# HOMO-LUMO Gap Predictions

### Problem Statement & Motivation

Accurately predicting quantum chemical properties like the HOMO–LUMO energy gap is essential for advancing materials science, drug discovery, and electronic design. The HOMO–LUMO gap is particularly informative for assessing molecular reactivity and stability. While Density Functional Theory (DFT) provides precise estimates, its high computational cost makes it impractical for large-scale screening of molecular libraries. This notebook explores machine learning alternatives that are fast, scalable, and interpretable, offering solutions that are accessible even on modest hardware.

### Related Work & Key Gap

Past work has shown that:

* DFT is accurate but computationally intensive
* ML models like kernel methods and GNNs show promise, but often require large models and expensive hardware

Key Gap: A need for lightweight, high-performing models that can run locally and integrate with user-friendly tools for deployment in research or education.

### Methodology & Evaluation

This notebook:

* Benchmarks a variety of 2D-based models using RDKit descriptors, Coulomb matrices, and graph neural networks (GNNs) on a 5k molecule subset
* Progresses to a hybrid GNN architecture combining OGB-standard graphs with SMILES-derived cheminformatics features
* Achieves **MAE = 0.159 eV**
* Visualizes results using parity plots, error inspection, and predicted-vs-true comparisons
* Evaluates both random and high-error cases to better understand model behavior

| Metric   | Best Model (Hybrid GNN) |
| -------- | ----------------------- |
| **MAE**  | 0.159 eV                |
| **RMSE** | 0.234 eV                |
| **R²**   | 0.965                   |


### Deployment & Accessibility

To make the model practically useful, an **interactive web app** was developed:

**Live App**: [HOMO–LUMO Gap Predictor on Hugging Face](https://huggingface.co/spaces/MooseML/homo-lumo-gap-predictor)

Features:

* **SMILES input** for any organic molecule
* **Real-time prediction** of the HOMO–LUMO gap
* **Molecular visualization**
* Simple **CSV logging** for result tracking

GitHub Repository: [MooseML/homo-lumo-gap-models](https://github.com/MooseML/homo-lumo-gap-models)


In [1]:
# general 
import pandas as pd
import numpy as np
from tqdm import tqdm
import ace_tools_open as tools
import optuna
import optuna.visualization as vis
import pickle
import joblib
import os 

# plotting 
import matplotlib.pyplot as plt
import seaborn as sns

# TensorFlow
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Add
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

# PyTorch
import torch
import torch.nn.functional as F
from torch.nn import Linear, ReLU, Module, Sequential, Dropout
from torch.utils.data import Subset
import torch.optim as optim
# PyTorch Geometric
from torch_geometric.nn import GINEConv, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from transformers import get_cosine_schedule_with_warmup

# OGB dataset 
from ogb.lsc import PygPCQM4Mv2Dataset, PCQM4Mv2Dataset
from ogb.utils import smiles2graph
from ogb.graphproppred.mol_encoder import AtomEncoder, BondEncoder

# RDKit
# from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit import Chem

# ChemML
from chemml.chem import Molecule, RDKitFingerprint, CoulombMatrix, tensorise_molecules
from chemml.models import MLP, NeuralGraphHidden, NeuralGraphOutput
from chemml.utils import regression_metrics

# SKlearn 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [2]:
print("TensorFlow version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("CUDA available:", tf.test.is_built_with_gpu_support())
print(tf.config.list_physical_devices('GPU'))
# list all GPUs
gpus = tf.config.list_physical_devices('GPU')

# check compute capability if GPU available
if gpus:
    for gpu in gpus:
        details = tf.config.experimental.get_device_details(gpu)
        print(f"Device: {gpu.name}")
        print(f"Compute Capability: {details.get('compute_capability')}")
else:
    print("No GPU found.")

TensorFlow version: 2.10.0
Built with CUDA: True
CUDA available: True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Device: /physical_device:GPU:0
Compute Capability: (8, 6)


In [3]:
# Paths - Fixed for Kaggle environment
if os.path.exists('/kaggle'):
    DATA_ROOT = '/kaggle/input/neurips-open-polymer-prediction-2025'
    CHUNK_DIR = '/kaggle/working/processed_chunks'  # Writable directory
    BACKBONE_PATH = '/kaggle/input/polymer/best_gnn_transformer_hybrid.pt'
else:
    DATA_ROOT = 'data'
    CHUNK_DIR = os.path.join(DATA_ROOT, 'processed_chunks')
    BACKBONE_PATH = 'best_gnn_transformer_hybrid.pt'

TRAIN_LMDB = os.path.join(CHUNK_DIR, 'polymer_train3d_dist.lmdb')
TEST_LMDB = os.path.join(CHUNK_DIR, 'polymer_test3d_dist.lmdb')

print(f"Data root: {DATA_ROOT}")
print(f"LMDB directory: {CHUNK_DIR}")
print(f"Train LMDB: {TRAIN_LMDB}")
print(f"Test LMDB: {TEST_LMDB}")

# Create LMDBs if they don't exist
if not os.path.exists(TRAIN_LMDB) or not os.path.exists(TEST_LMDB):
    print('Building LMDBs...')
    os.makedirs(CHUNK_DIR, exist_ok=True)
    # Run the LMDB builders
    !python build_polymer_lmdb_fixed.py train
    !python build_polymer_lmdb_fixed.py test
    print('LMDB creation complete.')
else:
    print('LMDBs already exist.')


Data root: data
LMDB directory: data\processed_chunks
Train LMDB: data\processed_chunks\polymer_train3d_dist.lmdb
Test LMDB: data\processed_chunks\polymer_test3d_dist.lmdb
LMDBs already exist.


In [4]:
train_path = os.path.join(DATA_ROOT, 'train.csv')
train_df   = pd.read_csv(train_path)

#  Keep only the columns we care about 
target_cols = ['SMILES', 'Tg', 'FFV', 'Tc', 'Density', 'Rg']
train_df = train_df[target_cols]        # drops id and any other columns

#  Sample a subset (optional) 
n = len(train_df)
subset_size = n                         # change to whatever you need
subset_df = train_df.sample(subset_size, random_state=42)

#  Save the subset as a CSV 
subset_path = os.path.join(DATA_ROOT, 'train_subset.csv')
subset_df.to_csv(subset_path, index=False)

print(f"Saved CSV with shape: {subset_df.shape}")
print(subset_df.head())

Saved CSV with shape: (7973, 6)
                                                 SMILES  Tg       FFV  \
7560  *C=Cc1ccc2c3ccc(*)cc3n(-c3ccc(OCCCCCCCCCC)c(OC... NaN  0.386695   
1405                  *CC(=O)NCCCCCCNC(=O)Cc1ccc(O*)cc1 NaN  0.335504   
5196                              *CC(*)c1ccccc1C(=O)NC NaN  0.355580   
2087  *c1ccc2c(c1)C(=O)N(c1ccc(Oc3ccc(N4C(=O)c5ccc(-... NaN  0.401573   
3337                    *CC(*)OC(=O)c1ccc(-c2ccccc2)cc1 NaN  0.353609   

            Tc  Density  Rg  
7560       NaN      NaN NaN  
1405       NaN      NaN NaN  
5196  0.183667      NaN NaN  
2087       NaN      NaN NaN  
3337       NaN      NaN NaN  


In [5]:
df = pd.read_csv(subset_path)
print(f"Loaded {len(df)} molecules.")

Loaded 7973 molecules.


The only property that appears will succeed with a simple imputation strategy is FFV. All other properties contain very high percent missing. Therefore, I will impute median for FFV, train a model for FFV, and train separate models for other properties. I will attempt to filter out missing values for each property. If this yields uncessful, I may explore sampling techniques or use the trained model to impute values to train a secondaery model. |

# Models

In [6]:
def build_target_df(df, target_col: str):
    """Return a DataFrame with only SMILES and the given target, dropping missing targets."""
    out = df[['SMILES', target_col]].copy()
    print(f"Initial {target_col} DataFrame shape:", out.shape)
    print(f"Initial {target_col} Missing Values:\n{out.isnull().sum()}")

    out = out.dropna(subset=[target_col]).reset_index(drop=True)

    print(f"\nCleaned {target_col} DataFrame shape:", out.shape)
    print(f"Cleaned {target_col} Missing Values:\n{out.isnull().sum()}")
    return out

# Build all five
df_tg      = build_target_df(df, 'Tg')
df_density = build_target_df(df, 'Density')
df_ffv     = build_target_df(df, 'FFV')   # per instructions: drop missing, no imputation
df_tc      = build_target_df(df, 'Tc')
df_rg      = build_target_df(df, 'Rg')


Initial Tg DataFrame shape: (7973, 2)
Initial Tg Missing Values:
SMILES       0
Tg        7462
dtype: int64

Cleaned Tg DataFrame shape: (511, 2)
Cleaned Tg Missing Values:
SMILES    0
Tg        0
dtype: int64
Initial Density DataFrame shape: (7973, 2)
Initial Density Missing Values:
SMILES        0
Density    7360
dtype: int64

Cleaned Density DataFrame shape: (613, 2)
Cleaned Density Missing Values:
SMILES     0
Density    0
dtype: int64
Initial FFV DataFrame shape: (7973, 2)
Initial FFV Missing Values:
SMILES      0
FFV       943
dtype: int64

Cleaned FFV DataFrame shape: (7030, 2)
Cleaned FFV Missing Values:
SMILES    0
FFV       0
dtype: int64
Initial Tc DataFrame shape: (7973, 2)
Initial Tc Missing Values:
SMILES       0
Tc        7236
dtype: int64

Cleaned Tc DataFrame shape: (737, 2)
Cleaned Tc Missing Values:
SMILES    0
Tc        0
dtype: int64
Initial Rg DataFrame shape: (7973, 2)
Initial Rg Missing Values:
SMILES       0
Rg        7359
dtype: int64

Cleaned Rg DataFrame sha

In [7]:
from rdkit.Chem import AllChem, Descriptors, HybridizationType, SanitizeFlags
def rdkit_ogb_agree(smi: str) -> bool:
    m = Chem.MolFromSmiles(smi)
    if m is None:
        return False
    return m.GetNumAtoms() == smiles2graph(smi)["num_nodes"]

def canonicalize_polymer_smiles(smiles: str, cap_atomic_num: int = 6) -> str:
    """
    Turn every '*' (dummy atom) into a real atom (default C) in the RDKit graph,
    preserving existing bond orders/stereo; sanitize, remove explicit Hs, and
    return canonical isomeric SMILES.
    """
    mol = Chem.MolFromSmiles(smiles, sanitize=False)
    if mol is None:
        raise ValueError(f"RDKit could not parse SMILES: {smiles}")

    rw = Chem.RWMol(mol)
    for a in rw.GetAtoms():
        if a.GetAtomicNum() == 0:   # '*'
            a.SetAtomicNum(cap_atomic_num)  # 6 = carbon
            a.SetFormalCharge(0)
            a.SetIsAromatic(False)
            a.SetNoImplicit(False)
            a.SetNumExplicitHs(0)

    mol2 = rw.GetMol()
    try:
        Chem.SanitizeMol(mol2)
    except Exception:
        Chem.SanitizeMol(mol2, sanitizeOps=SanitizeFlags.SANITIZE_ALL ^ SanitizeFlags.SANITIZE_KEKULIZE)
        Chem.Kekulize(mol2, clearAromaticFlags=True)

    mol2 = Chem.RemoveHs(mol2)
    return Chem.MolToSmiles(mol2, isomericSmiles=True, canonical=True)

In [8]:
from typing import Optional, Tuple, List

def prepare_target_data(
    df_target: pd.DataFrame,
    target_col: str,
    *,
    do_3d: bool = True,
    optimizer: str = "MMFF",
    max_iters: int = 200,
    add_hydrogens: bool = True,
    fp_type: str = "morgan",
    fp_vector: str = "bit",
    fp_bits: int = 1024,
    fp_radius: int = 3,
    save_csv_path: Optional[str] = None,
    show_progress: bool = True,
) -> Tuple[pd.DataFrame, List, np.ndarray, np.ndarray]:
    """
    Build molecules, drop missing targets, compute RDKit fingerprints.
    Returns: (df_clean, valid_mol_objs, y, X_fp)

    df_clean columns: ['SMILES', target_col]
    y shape: (N,)
    X_fp shape: (N, fp_bits) for bit vector
    """
    assert {"SMILES", target_col}.issubset(df_target.columns), "df_target must have SMILES and target column"

    # 1) Drop rows with missing targets (no label imputation)
    df_work = df_target[["SMILES", target_col]].copy()
    before = len(df_work)
    df_work = df_work.dropna(subset=[target_col]).reset_index(drop=True)
    after = len(df_work)
    print(f"[{target_col}] dropped {before - after} rows with missing target; kept {after}")

    valid_mol_objs: List = []
    valid_targets: List[float] = []

    it = df_work.itertuples(index=False)
    if show_progress:
        it = tqdm(it, total=len(df_work), desc=f"Build 3D + collect {target_col}")

    for row in it:
        smi = row.SMILES
        tval = getattr(row, target_col)

        try:
            clean = canonicalize_polymer_smiles(smi)
            if not isinstance(clean, str) or not clean:
                continue

            mol = Molecule(clean, input_type="smiles")
            if add_hydrogens:
                mol.hydrogens("add")
            if do_3d:
                mol.to_xyz(optimizer=optimizer, maxIters=max_iters)
                if mol.xyz is None:
                    continue

            valid_mol_objs.append(mol)
            valid_targets.append(float(tval))

        except Exception as e:
            # skip problematic molecules; keep going
            # print(f"Skipped {smi}: {e}")  # uncomment if you want verbose logging
            continue

    print(f"[{target_col}] kept {len(valid_mol_objs)} molecules after 3D/cleaning.")

    # 2) Build cleaned DataFrame and target vector
    df_clean = pd.DataFrame({
        "SMILES": [m.smiles for m in valid_mol_objs],
        target_col: np.array(valid_targets, dtype=float)
    })
    if save_csv_path:
        df_clean.to_csv(save_csv_path, index=False)
        print(f"[{target_col}] saved cleaned dataset -> {save_csv_path}")

    y = np.asarray(valid_targets, dtype=float)

    # 3) Compute RDKit fingerprint features
    fp_featurizer = RDKitFingerprint(
        fingerprint_type=fp_type,
        vector=fp_vector,     # 'bit' or 'count'
        n_bits=fp_bits,
        radius=fp_radius
    )
    X_fp = fp_featurizer.represent(valid_mol_objs)

    print(f"[{target_col}] X_fp shape: {X_fp.shape} | y shape: {y.shape}")
    return df_clean, valid_mol_objs, y, X_fp


In [9]:
# Tg
df_clean_tg, mols_tg, y_tg, X_tg = prepare_target_data(
    df_tg, "Tg",
    do_3d=True, optimizer="MMFF", max_iters=200,
    fp_type="morgan", fp_vector="bit", fp_bits=1024, fp_radius=3,
    save_csv_path="cleaned_tg_dataset.csv"
)

# Density
df_clean_density, mols_density, y_density, X_density = prepare_target_data(
    df_density, "Density",
    do_3d=True, optimizer="MMFF", max_iters=200,
    fp_type="morgan", fp_vector="bit", fp_bits=1024, fp_radius=3,
    save_csv_path="cleaned_density_dataset.csv"
)

# FFV
# If you want to skip 3D for a tabular-first target:
df_clean_ffv, mols_ffv, y_ffv, X_ffv = prepare_target_data(
    df_ffv, "FFV", 
    do_3d=False,
    save_csv_path="cleaned_ffv_dataset.csv")


# Tc
df_clean_tc, mols_tc, y_tc, X_tc = prepare_target_data(
    df_tc, "Tc",
    do_3d=True, optimizer="MMFF", max_iters=200,
    fp_type="morgan", fp_vector="bit", fp_bits=1024, fp_radius=3,
    save_csv_path="cleaned_tc_dataset.csv"
)

# Rg
df_clean_rg, mols_rg, y_rg, X_rg = prepare_target_data(
    df_rg, "Rg",
    do_3d=True, optimizer="MMFF", max_iters=200,
    fp_type="morgan", fp_vector="bit", fp_bits=1024, fp_radius=3,
    save_csv_path="cleaned_rg_dataset.csv"
)


[Tg] dropped 0 rows with missing target; kept 511


Build 3D + collect Tg: 100%|██████████| 511/511 [00:35<00:00, 14.48it/s]


[Tg] kept 504 molecules after 3D/cleaning.
[Tg] saved cleaned dataset -> cleaned_tg_dataset.csv
[Tg] X_fp shape: (504, 1024) | y shape: (504,)
[Density] dropped 0 rows with missing target; kept 613


Build 3D + collect Density: 100%|██████████| 613/613 [00:35<00:00, 17.17it/s]


[Density] kept 609 molecules after 3D/cleaning.
[Density] saved cleaned dataset -> cleaned_density_dataset.csv
[Density] X_fp shape: (609, 1024) | y shape: (609,)
[FFV] dropped 0 rows with missing target; kept 7030


...

[FFV] kept 7030 molecules after 3D/cleaning.
[FFV] saved cleaned dataset -> cleaned_ffv_dataset.csv
[FFV] X_fp shape: (7030, 1024) | y shape: (7030,)
[Tc] dropped 0 rows with missing target; kept 737


Build 3D + collect Tc: 100%|██████████| 737/737 [00:41<00:00, 17.67it/s]


[Tc] kept 734 molecules after 3D/cleaning.
[Tc] saved cleaned dataset -> cleaned_tc_dataset.csv
[Tc] X_fp shape: (734, 1024) | y shape: (734,)
[Rg] dropped 0 rows with missing target; kept 614


Build 3D + collect Rg: 100%|██████████| 614/614 [00:34<00:00, 17.60it/s]


[Rg] kept 611 molecules after 3D/cleaning.
[Rg] saved cleaned dataset -> cleaned_rg_dataset.csv
[Rg] X_fp shape: (611, 1024) | y shape: (611,)


In [10]:
from dataclasses import dataclass
from typing import Optional, Tuple
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

@dataclass
class TabularSplits:
    # unscaled (use for RF)
    X_train: np.ndarray
    X_test:  np.ndarray
    y_train: np.ndarray
    y_test:  np.ndarray
    # scaled (use for KRR/MLP)
    X_train_scaled: Optional[np.ndarray] = None
    X_test_scaled:  Optional[np.ndarray] = None
    y_train_scaled: Optional[np.ndarray] = None  # shape (N,1)
    y_test_scaled:  Optional[np.ndarray] = None
    x_scaler: Optional[StandardScaler] = None
    y_scaler: Optional[StandardScaler] = None

def _make_regression_stratify_bins(y: np.ndarray, n_bins: int = 10) -> np.ndarray:
    """Return integer bins for approximate stratification in regression."""
    y = y.ravel()
    # handle degenerate case
    if np.unique(y).size < n_bins:
        n_bins = max(2, np.unique(y).size)
    quantiles = np.linspace(0, 1, n_bins + 1)
    bins = np.unique(np.quantile(y, quantiles))
    # ensure strictly increasing
    bins = np.unique(bins)
    # np.digitize expects right-open intervals by default
    strat = np.digitize(y, bins[1:-1], right=False)
    return strat

def make_tabular_splits(
    X: np.ndarray,
    y: np.ndarray,
    *,
    test_size: float = 0.2,
    random_state: int = 42,
    scale_X: bool = True,
    scale_y: bool = True,
    stratify_regression: bool = False,
    n_strat_bins: int = 10,
    # if you already decided splits (e.g., scaffold split), pass indices:
    train_idx: Optional[np.ndarray] = None,
    test_idx: Optional[np.ndarray] = None,
) -> TabularSplits:
    """
    Split and (optionally) scale tabular features/targets for a single target.
    Returns both scaled and unscaled arrays, plus fitted scalers.
    """
    y = np.asarray(y, dtype=float).ravel()
    X = np.asarray(X)

    if train_idx is not None and test_idx is not None:
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
    else:
        strat = None
        if stratify_regression:
            strat = _make_regression_stratify_bins(y, n_bins=n_strat_bins)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, stratify=strat
        )

    # Unscaled outputs (for RF, tree models)
    splits = TabularSplits(
        X_train=X_train, X_test=X_test,
        y_train=y_train, y_test=y_test
    )

    # Scaled versions (for KRR/MLP)
    if scale_X:
        xscaler = StandardScaler()
        splits.X_train_scaled = xscaler.fit_transform(X_train)
        splits.X_test_scaled  = xscaler.transform(X_test)
        splits.x_scaler = xscaler
    if scale_y:
        yscaler = StandardScaler()
        splits.y_train_scaled = yscaler.fit_transform(y_train.reshape(-1, 1))
        splits.y_test_scaled  = yscaler.transform(y_test.reshape(-1, 1))
        splits.y_scaler = yscaler

    # Shapes summary
    print("Splits:")
    print("  X_train:", splits.X_train.shape, "| X_test:", splits.X_test.shape)
    if splits.X_train_scaled is not None:
        print("  X_train_scaled:", splits.X_train_scaled.shape, "| X_test_scaled:", splits.X_test_scaled.shape)
    print("  y_train:", splits.y_train.shape, "| y_test:", splits.y_test.shape)
    if splits.y_train_scaled is not None:
        print("  y_train_scaled:", splits.y_train_scaled.shape, "| y_test_scaled:", splits.y_test_scaled.shape)

    return splits

In [11]:
from typing import Dict, Any, Tuple
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import numpy as np
import os

def train_eval_rf(
    X: np.ndarray,
    y: np.ndarray,
    *,
    rf_params: Dict[str, Any],
    test_size: float = 0.2,
    random_state: int = 42,
    stratify_regression: bool = True,
    n_strat_bins: int = 10,
    save_dir: str = "saved_models/rf",
    tag: str = "model",
) -> Tuple[RandomForestRegressor, Dict[str, float], TabularSplits, str]:
    """
    Trains a RandomForest on unscaled features; returns (model, metrics, splits, path).
    """
    os.makedirs(save_dir, exist_ok=True)

    splits = make_tabular_splits(
        X, y,
        test_size=test_size,
        random_state=random_state,
        scale_X=False, scale_y=False,                 # RF doesn't need scaling
        stratify_regression=stratify_regression,
        n_strat_bins=n_strat_bins
    )

    rf = RandomForestRegressor(random_state=random_state, n_jobs=-1, **rf_params)
    rf.fit(splits.X_train, splits.y_train)

    pred_tr = rf.predict(splits.X_train)
    pred_te = rf.predict(splits.X_test)

    metrics = {
        "train_MAE": mean_absolute_error(splits.y_train, pred_tr),
        "train_RMSE": mean_squared_error(splits.y_train, pred_tr, squared=False),
        "train_R2": r2_score(splits.y_train, pred_tr),
        "val_MAE": mean_absolute_error(splits.y_test, pred_te),
        "val_RMSE": mean_squared_error(splits.y_test, pred_te, squared=False),
        "val_R2": r2_score(splits.y_test, pred_te),
    }
    print(f"[RF/{tag}] val_MAE={metrics['val_MAE']:.6f}  val_RMSE={metrics['val_RMSE']:.6f}  val_R2={metrics['val_R2']:.4f}")

    path = os.path.join(save_dir, f"rf_{tag}.joblib")
    joblib.dump({"model": rf, "metrics": metrics, "rf_params": rf_params}, path)
    return rf, metrics, splits, path


In [12]:
rf_cfg = {
    "FFV": {"n_estimators": 100, "max_depth": 60},
    "Tc":  {'n_estimators': 800, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': False},
    "Rg":  {'n_estimators': 400, 'max_depth': 260, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_features': 1.0, 'bootstrap': True},
}

rf_ffv, m_ffv, splits_ffv, p_ffv = train_eval_rf(X_ffv, y_ffv, rf_params=rf_cfg["FFV"], tag="FFV")
rf_tc,  m_tc,  splits_tc,  p_tc  = train_eval_rf(X_tc,  y_tc,  rf_params=rf_cfg["Tc"],  tag="Tc")
rf_rg,  m_rg,  splits_rg,  p_rg  = train_eval_rf(X_rg,  y_rg,  rf_params=rf_cfg["Rg"],  tag="Rg")


Splits:
  X_train: (5624, 1024) | X_test: (1406, 1024)
  y_train: (5624,) | y_test: (1406,)
[RF/FFV] val_MAE=0.008940  val_RMSE=0.015961  val_R2=0.7148
Splits:
  X_train: (587, 1024) | X_test: (147, 1024)
  y_train: (587,) | y_test: (147,)
[RF/Tc] val_MAE=0.030779  val_RMSE=0.043855  val_R2=0.7543
Splits:
  X_train: (488, 1024) | X_test: (123, 1024)
  y_train: (488,) | y_test: (123,)
[RF/Rg] val_MAE=2.042348  val_RMSE=2.931961  val_R2=0.6087


In [13]:
# If needed:
# !pip install -q optuna

import optuna
import numpy as np
import joblib, os
from typing import Dict, Any, Optional, Tuple
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def _rmse_safe(y_true, y_pred):
    # works on older sklearn (no squared=)
    try:
        return mean_squared_error(y_true, y_pred, squared=False)
    except TypeError:
        return np.sqrt(mean_squared_error(y_true, y_pred))

def optuna_tune_extratrees(
    X: np.ndarray,
    y: np.ndarray,
    *,
    n_trials: int = 60,
    timeout: Optional[int] = None,     # seconds; or None
    test_size: float = 0.2,
    random_state: int = 42,
    stratify_regression: bool = True,
    n_strat_bins: int = 10,
    metric: str = "mae",               # "mae" | "rmse" | "mse" | "r2" (maximize r2)
    save_dir: str = "saved_models/et_optuna",
    tag: str = "model",
    verbose: bool = True,
) -> Tuple[ExtraTreesRegressor, Dict[str, Any], Dict[str, float], str, optuna.Study]:
    """
    Runs Optuna to tune ExtraTreesRegressor on a single holdout split.
    Returns (best_model, best_params, best_metrics, path_to_joblib, study).
    """
    os.makedirs(save_dir, exist_ok=True)
    # One fixed split for fair comparison with your RF helper
    splits = make_tabular_splits(
        X, y,
        test_size=test_size,
        random_state=random_state,
        scale_X=False, scale_y=False,
        stratify_regression=stratify_regression,
        n_strat_bins=n_strat_bins
    )

    def objective(trial: optuna.Trial):
        # Search space (kept tight to be Kaggle-friendly)
        params = {
            "n_estimators":      trial.suggest_int("n_estimators", 400, 1400, step=100),
            "max_depth":         trial.suggest_categorical("max_depth", [None] + list(range(20, 121, 10))),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
            "min_samples_leaf":  trial.suggest_int("min_samples_leaf", 1, 10),
            "max_features":      trial.suggest_categorical("max_features", [1.0, 0.9, 0.7, "sqrt"]),
            "bootstrap":         trial.suggest_categorical("bootstrap", [False, True]),
            "criterion":         trial.suggest_categorical("criterion", ["squared_error", "absolute_error"]),
            "random_state":      random_state,
            "n_jobs":            -1,
        }

        model = ExtraTreesRegressor(**params)
        model.fit(splits.X_train, splits.y_train)

        pred_tr = model.predict(splits.X_train)
        pred_te = model.predict(splits.X_test)

        # Compute metrics on VAL split
        mse  = mean_squared_error(splits.y_test, pred_te)
        rmse = np.sqrt(mse)
        mae  = mean_absolute_error(splits.y_test, pred_te)
        r2   = r2_score(splits.y_test, pred_te)

        # Choose objective
        if metric == "mae":
            score = mae
        elif metric == "rmse":
            score = rmse
        elif metric == "mse":
            score = mse
        elif metric == "r2":
            score = -r2  # maximize R2 by minimizing -R2
        else:
            score = mae  # default

        # Report for potential pruning (single step)
        trial.report(score, step=0)
        return score

    study = optuna.create_study(direction="minimize", study_name=f"ET_{tag}")
    study.optimize(objective, n_trials=n_trials, timeout=timeout, show_progress_bar=verbose)

    best_params = study.best_params
    # Refit best model on the SAME training split (you can also refit on all data later)
    best_model = ExtraTreesRegressor(**{
        **best_params, "random_state": random_state, "n_jobs": -1
    }).fit(splits.X_train, splits.y_train)

    # Final validation metrics for the chosen params
    pred_tr = best_model.predict(splits.X_train)
    pred_te = best_model.predict(splits.X_test)
    best_metrics = {
        "train_MAE":  mean_absolute_error(splits.y_train, pred_tr),
        "train_RMSE": _rmse_safe(splits.y_train, pred_tr),
        "train_R2":   r2_score(splits.y_train, pred_tr),
        "val_MAE":    mean_absolute_error(splits.y_test,  pred_te),
        "val_RMSE":   _rmse_safe(splits.y_test,  pred_te),
        "val_R2":     r2_score(splits.y_test,  pred_te),
    }
    if verbose:
        print(f"[ET/Optuna/{tag}] best params: {best_params}")
        print(f"[ET/Optuna/{tag}] val_MAE={best_metrics['val_MAE']:.6f}  "
              f"val_RMSE={best_metrics['val_RMSE']:.6f}  val_R2={best_metrics['val_R2']:.4f}")

    path = os.path.join(save_dir, f"et_{tag}_best.joblib")
    joblib.dump({"model": best_model, "params": best_params, "metrics": best_metrics}, path)
    return best_model, best_params, best_metrics, path, study


# FFV (lots of data → good candidate)
best_et_ffv, bp_ffv, met_ffv, p_et_ffv, study_ffv = optuna_tune_extratrees(
    X_ffv, y_ffv, n_trials=60, metric="mae", tag="FFV"
)

# Tc
best_et_tc, bp_tc, met_tc, p_et_tc, study_tc = optuna_tune_extratrees(
    X_tc, y_tc, n_trials=60, metric="mae", tag="Tc"
)

# Rg
best_et_rg, bp_rg, met_rg, p_et_rg, study_rg = optuna_tune_extratrees(
    X_rg, y_rg, n_trials=60, metric="mae", tag="Rg"
)

[I 2025-09-06 08:44:29,910] A new study created in memory with name: ET_FFV


Splits:
  X_train: (5624, 1024) | X_test: (1406, 1024)
  y_train: (5624,) | y_test: (1406,)


  0%|          | 0/60 [00:00<?, ?it/s]

[I 2025-09-06 08:44:30,717] Trial 0 finished with value: 0.010924571875946127 and parameters: {'n_estimators': 500, 'max_depth': 60, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'bootstrap': True, 'criterion': 'squared_error'}. Best is trial 0 with value: 0.010924571875946127.
[I 2025-09-06 08:44:43,810] Trial 1 finished with value: 0.009919091298697894 and parameters: {'n_estimators': 1100, 'max_depth': 90, 'min_samples_split': 12, 'min_samples_leaf': 7, 'max_features': 0.9, 'bootstrap': True, 'criterion': 'squared_error'}. Best is trial 1 with value: 0.009919091298697894.
[I 2025-09-06 08:45:04,622] Trial 2 finished with value: 0.009391311156532672 and parameters: {'n_estimators': 1100, 'max_depth': 30, 'min_samples_split': 13, 'min_samples_leaf': 1, 'max_features': 1.0, 'bootstrap': True, 'criterion': 'squared_error'}. Best is trial 2 with value: 0.009391311156532672.
[I 2025-09-06 08:45:05,452] Trial 3 finished with value: 0.012489161825258601 and paramete

[I 2025-09-06 15:00:12,369] A new study created in memory with name: ET_Tc


...

[I 2025-09-06 15:00:13,004] Trial 0 finished with value: 0.03505347611730708 and parameters: {'n_estimators': 1000, 'max_depth': 120, 'min_samples_split': 2, 'min_samples_leaf': 10, 'max_features': 'sqrt', 'bootstrap': False, 'criterion': 'squared_error'}. Best is trial 0 with value: 0.03505347611730708.
[I 2025-09-06 15:00:13,530] Trial 1 finished with value: 0.03486280541383221 and parameters: {'n_estimators': 400, 'max_depth': 70, 'min_samples_split': 3, 'min_samples_leaf': 10, 'max_features': 'sqrt', 'bootstrap': False, 'criterion': 'absolute_error'}. Best is trial 1 with value: 0.03486280541383221.
[I 2025-09-06 15:00:16,280] Trial 2 finished with value: 0.03147888819241985 and parameters: {'n_estimators': 1000, 'max_depth': 70, 'min_samples_split': 5, 'min_samples_leaf': 10, 'max_features': 0.7, 'bootstrap': True, 'criterion': 'absolute_error'}. Best is trial 2 with value: 0.03147888819241985.
[I 2025-09-06 15:00:17,794] Trial 3 finished with value: 0.03240693206265789 and parame

[I 2025-09-06 15:09:41,751] A new study created in memory with name: ET_Rg


[ET/Optuna/Tc] best params: {'n_estimators': 800, 'max_depth': 110, 'min_samples_split': 10, 'min_samples_leaf': 3, 'max_features': 0.9, 'bootstrap': True, 'criterion': 'absolute_error'}
[ET/Optuna/Tc] val_MAE=0.029706  val_RMSE=0.043069  val_R2=0.7630
Splits:
  X_train: (488, 1024) | X_test: (123, 1024)
  y_train: (488,) | y_test: (123,)


  0%|          | 0/60 [00:00<?, ?it/s]

[I 2025-09-06 15:09:45,459] Trial 0 finished with value: 2.1853420373234624 and parameters: {'n_estimators': 1100, 'max_depth': 100, 'min_samples_split': 9, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'bootstrap': True, 'criterion': 'absolute_error'}. Best is trial 0 with value: 2.1853420373234624.
[I 2025-09-06 15:09:53,965] Trial 1 finished with value: 2.192977698872571 and parameters: {'n_estimators': 1200, 'max_depth': 30, 'min_samples_split': 18, 'min_samples_leaf': 8, 'max_features': 0.7, 'bootstrap': False, 'criterion': 'absolute_error'}. Best is trial 0 with value: 2.1853420373234624.
[I 2025-09-06 15:09:55,238] Trial 2 finished with value: 2.4298811882841673 and parameters: {'n_estimators': 1100, 'max_depth': 50, 'min_samples_split': 6, 'min_samples_leaf': 9, 'max_features': 1.0, 'bootstrap': False, 'criterion': 'squared_error'}. Best is trial 0 with value: 2.1853420373234624.
[I 2025-09-06 15:09:57,255] Trial 3 finished with value: 2.020266916122951 and parameters: {'n_est

In [14]:
from typing import Dict, Any, Tuple
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np, joblib, os

def _rmse_safe(y_true, y_pred):
    # Kaggle-safe RMSE (handles older sklearn)
    try:
        return mean_squared_error(y_true, y_pred, squared=False)
    except TypeError:
        return np.sqrt(mean_squared_error(y_true, y_pred))

def train_eval_extratrees(
    X: np.ndarray,
    y: np.ndarray,
    *,
    et_params: Dict[str, Any],
    test_size: float = 0.2,
    random_state: int = 42,
    stratify_regression: bool = True,
    n_strat_bins: int = 10,
    save_dir: str = "saved_models/et",
    tag: str = "model",
) -> Tuple[ExtraTreesRegressor, Dict[str, float], "TabularSplits", str]:
    """
    Trains an ExtraTreesRegressor on unscaled features; returns (model, metrics, splits, path).
    """
    os.makedirs(save_dir, exist_ok=True)

    splits = make_tabular_splits(
        X, y,
        test_size=test_size,
        random_state=random_state,
        scale_X=False, scale_y=False,
        stratify_regression=stratify_regression,
        n_strat_bins=n_strat_bins
    )

    params = dict(et_params) if et_params else {}
    params.setdefault("random_state", random_state)
    params.setdefault("n_jobs", -1)

    et = ExtraTreesRegressor(**params)
    et.fit(splits.X_train, splits.y_train)

    pred_tr = et.predict(splits.X_train)
    pred_te = et.predict(splits.X_test)

    metrics = {
        "train_MAE":  mean_absolute_error(splits.y_train, pred_tr),
        "train_RMSE": _rmse_safe(splits.y_train, pred_tr),
        "train_R2":   r2_score(splits.y_train, pred_tr),
        "val_MAE":    mean_absolute_error(splits.y_test,  pred_te),
        "val_RMSE":   _rmse_safe(splits.y_test,  pred_te),
        "val_R2":     r2_score(splits.y_test,  pred_te),
    }
    print(f"[ET/{tag}] val_MAE={metrics['val_MAE']:.6f}  val_RMSE={metrics['val_RMSE']:.6f}  val_R2={metrics['val_R2']:.4f}")

    path = os.path.join(save_dir, f"et_{tag}.joblib")
    joblib.dump({"model": et, "metrics": metrics, "params": params}, path)
    return et, metrics, splits, path


In [15]:
et_cfg = {
    "FFV": {
        "n_estimators": 1000, "max_depth": None,
        "min_samples_split": 2, "min_samples_leaf": 1,
        "max_features": 1.0,   # <- often strong for ExtraTrees
        "bootstrap": False,
    },
    "Tc": {
        "n_estimators": 800, "max_depth": 40,
        "min_samples_split": 2, "min_samples_leaf": 1,
        "max_features": 0.9,
        "bootstrap": False,
    },
    "Rg": {
        "n_estimators": 1200, "max_depth": 60,
        "min_samples_split": 2, "min_samples_leaf": 2,  # a touch more regularization
        "max_features": 1.0,
        "bootstrap": False,
    },
}

et_ffv, m_et_ffv, splits_et_ffv, p_et_ffv = train_eval_extratrees(X_ffv, y_ffv, et_params=et_cfg["FFV"], tag="FFV")
et_tc,  m_et_tc,  splits_et_tc,  p_et_tc  = train_eval_extratrees(X_tc,  y_tc,  et_params=et_cfg["Tc"],  tag="Tc")
et_rg,  m_et_rg,  splits_et_rg,  p_et_rg  = train_eval_extratrees(X_rg,  y_rg,  et_params=et_cfg["Rg"],  tag="Rg")


Splits:
  X_train: (5624, 1024) | X_test: (1406, 1024)
  y_train: (5624,) | y_test: (1406,)
[ET/FFV] val_MAE=0.011200  val_RMSE=0.020049  val_R2=0.5501
Splits:
  X_train: (587, 1024) | X_test: (147, 1024)
  y_train: (587,) | y_test: (147,)
[ET/Tc] val_MAE=0.036053  val_RMSE=0.056604  val_R2=0.5906
Splits:
  X_train: (488, 1024) | X_test: (123, 1024)
  y_train: (488,) | y_test: (123,)
[ET/Rg] val_MAE=2.306750  val_RMSE=3.373014  val_R2=0.4821


## ChemML GNN Model Results
| Model Type             | Featurization        |   MAE |  RMSE |   R² | Notes             |
|------------------------|----------------------|-------|-------|------|-------------------|
| GNN (Tuned)            | tensorise_molecules Graph   | 0.302 | 0.411 | 0.900 | Best performance across all metrics   |
| GNN (Untuned)          | tensorise_molecules Graph   | 0.400 | 0.519 | 0.841 | Good overall|


---
# Final Model Training

Having explored different molecular graph representations and model architectures, I am now moving to training what is expected to be the best-performing model using the full dataset. The earlier GNN model was based on `tensorise_molecules` (ChemML) graphs and had strong performance with a **mean absolute error (MAE) around 0.30**. These graphs are based on RDKit's internal descriptors and do not reflect the original PCQM4Mv2 graph structure used in the Open Graph Benchmark (OGB). Therefore, I will shift focus to the `smiles2graph` representation provided by OGB, which aligns more directly with the benchmark's evaluation setup and top-performing models on the leaderboard.


| Source                         | Atom/Bond Features                                                 | Format                                          | Customizable?     | Alignment with PCQM4Mv2?  |
| ------------------------------ | ------------------------------------------------------------------ | ----------------------------------------------- | ----------------- | ---------------------- |
| `tensorise_molecules` (ChemML) | RDKit-based descriptors (ex: atom number, degree, hybridization) | NumPy tensors (`X_atoms`, `X_bonds`, `X_edges`) | Limited           |  Not aligned          |
| `smiles2graph` (OGB / PyG)     | Predefined categorical features from PCQM4Mv2                      | PyTorch Geometric `Data` objects                |  Highly flexible |  Matches OGB standard |

By using `smiles2graph`, we:

* Use OGB-standard graph construction and feature encoding for fair comparisons with leaderboard models
* Include learnable AtomEncoder and BondEncoder embeddings from `ogb.graphproppred.mol_encoder`, which improve model expressiveness
* Maintain compatibility with PyTorch Geometric, DGL, and OGB tools

I will also concatenate GNN-derived embeddings with SMILES-based RDKit descriptors, feeding this hybrid representation into MLP head. This allows you to combine structural and cheminformatics perspectives for improved prediction accuracy. With this setup, I aim to improve upon the MAE of \~0.30 achieved earlier and push closer toward state-of-the-art performance.




In [ ]:
from rdkit.Chem import Descriptors, rdMolDescriptors as rdmd
# def compute_rdkit_descriptors(smiles: str):
#     s = canonicalize_polymer_smiles(smiles)
#     m = Chem.MolFromSmiles(s)
#     if m is None or m.GetNumAtoms() == 0:
#         return [np.nan]*9
#     return [
#         Descriptors.MolWt(m),
#         Descriptors.NumRotatableBonds(m),
#         Descriptors.TPSA(m),
#         Descriptors.NumHAcceptors(m),
#         Descriptors.NumHDonors(m),
#         Descriptors.RingCount(m),
#         Descriptors.FractionCSP3(m),
#         Descriptors.MolLogP(m),
#         Descriptors.NumSaturatedRings(m),
#     ]


def compute_rdkit_descriptors(m):
    s = canonicalize_polymer_smiles(m)
    m = Chem.MolFromSmiles(s)
    heavy = Descriptors.HeavyAtomCount(m)
    arom_atoms = sum(a.GetIsAromatic() for a in m.GetAtoms())
    hetero = sum(a.GetAtomicNum() not in (1,6) for a in m.GetAtoms())
    aromatic_prop = (arom_atoms / max(1, heavy))
    hetero_frac   = (hetero / max(1, heavy))

    return [
        Descriptors.MolMR(m),                    # 1
        Descriptors.MolLogP(m),                  # 2 (SlogP)
        rdmd.CalcFractionCSP3(m),                # 3
        rdmd.CalcNumAromaticRings(m),            # 4
        aromatic_prop,                           # 5
        Descriptors.TPSA(m),                     # 6
        Descriptors.NumRotatableBonds(m),        # 7
        hetero_frac,                             # 8
        rdmd.CalcLabuteASA(m),                   # 9
        # rdmd.CalcBalabanJ(m),                    # 10
        rdmd.CalcKappa1(m),                      # 11
        rdmd.CalcKappa2(m),                      # 12
    ]




In [35]:
# -- Tg --
df_tg = pd.read_csv('cleaned_tg_dataset.csv')
smiles_tg = df_tg['SMILES'].tolist()
y_tg = df_tg['Tg'].to_numpy(dtype=float)

X_tg_desc = np.array([compute_rdkit_descriptors(s) for s in smiles_tg])
mask_tg = ~np.isnan(X_tg_desc).any(axis=1)
smiles_tg = list(np.array(smiles_tg)[mask_tg])
y_tg = y_tg[mask_tg]
X_tg_desc = X_tg_desc[mask_tg]
print("Tg descriptors:", X_tg_desc.shape, "labels:", y_tg.shape)

# -- Density --
df_density = pd.read_csv('cleaned_density_dataset.csv')
smiles_density = df_density['SMILES'].tolist()
y_density = df_density['Density'].to_numpy(dtype=float)

X_density_desc = np.array([compute_rdkit_descriptors(s) for s in smiles_density])
mask_den = ~np.isnan(X_density_desc).any(axis=1)
smiles_density = list(np.array(smiles_density)[mask_den])
y_density = y_density[mask_den]
X_density_desc = X_density_desc[mask_den]
print("Density descriptors:", X_density_desc.shape, "labels:", y_density.shape)


Tg descriptors: (504, 6) labels: (504,)
Density descriptors: (609, 6) labels: (609,)


In [36]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as GeoDataLoader
from sklearn.model_selection import train_test_split

def build_pyg_dataset(smiles: list, X_desc: np.ndarray, y: np.ndarray) -> list:
    assert len(smiles) == len(y) == X_desc.shape[0]
    out = []
    for i, smi in enumerate(smiles):
        g = smiles2graph(smi)
        out.append(Data(
            x=torch.tensor(g['node_feat'], dtype=torch.long),
            edge_index=torch.tensor(g['edge_index'], dtype=torch.long),
            edge_attr=torch.tensor(g['edge_feat'], dtype=torch.long),
            rdkit_feats=torch.tensor(X_desc[i], dtype=torch.float32),
            y=torch.tensor([y[i]], dtype=torch.float32),
        ))
    return out

def make_loaders(data_list, test_size=0.2, batch_size=32, seed=42):
    tr, va = train_test_split(data_list, test_size=test_size, random_state=seed)
    return (
        GeoDataLoader(tr, batch_size=batch_size, shuffle=True,  num_workers=0, pin_memory=True),
        GeoDataLoader(va, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True),
    )

data_tg = build_pyg_dataset(smiles_tg, X_tg_desc, y_tg)
train_loader_tg, val_loader_tg = make_loaders(data_tg, test_size=0.2, batch_size=32, seed=42)

data_density = build_pyg_dataset(smiles_density, X_density_desc, y_density)
train_loader_den, val_loader_den = make_loaders(data_density, test_size=0.2, batch_size=32, seed=42)


## Step 5: Define the Hybrid GNN Model

The final architecture uses both structural and cheminformatics data by combining GNN-learned graph embeddings with SMILES-derived RDKit descriptors. This Hybrid GNN model uses `smiles2graph` for graph construction and augments it with RDKit-based molecular features for improved prediction accuracy.

### Model Components:

* **AtomEncoder / BondEncoder**
  Transforms categorical atom and bond features (provided by OGB) into learnable embeddings using the encoders from `ogb.graphproppred.mol_encoder`. These provide a strong foundation for expressive graph learning.

* **GINEConv Layers (x2)**
  I use two stacked GINEConv layers (Graph Isomorphism Network with Edge features). These layers perform neighborhood aggregation based on edge attributes, allowing the model to capture localized chemical environments.

* **Global Mean Pooling**
  After message passing, node level embeddings are aggregated into a fixed size graph level representation using `global_mean_pool`.

* **Concatenation with RDKit Descriptors**
  The pooled GNN embedding is concatenated with external RDKit descriptors, which capture global molecular properties not easily inferred from graph data alone.

* **MLP Prediction Head**
  A multilayer perceptron processes the combined feature vector with ReLU activations, dropout regularization, and linear layers to predict the HOMO–LUMO gap.

In [37]:
import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as GeoDataLoader
from sklearn.model_selection import train_test_split
from typing import List


def _act(name: str):
    name = (name or "ReLU").lower()
    if name in ("relu",):   return nn.ReLU()
    if name in ("gelu",):   return nn.GELU()
    if name in ("swish","silu"): return nn.SiLU()
    return nn.ReLU()

class HybridGNN(Module):
    def __init__(self, gnn_dim: int, rdkit_dim: int, hidden_dim: int, dropout_rate: float=0.2, activation: str="ReLU"):
        super().__init__()
        self.gnn_dim = gnn_dim
        self.rdkit_dim = rdkit_dim
        act = _act(activation)
        self.atom_encoder = AtomEncoder(emb_dim=gnn_dim)
        self.bond_encoder = BondEncoder(emb_dim=gnn_dim)

        self.conv1 = GINEConv(Sequential(Linear(gnn_dim, gnn_dim), act, Linear(gnn_dim, gnn_dim)))
        self.conv2 = GINEConv(Sequential(Linear(gnn_dim, gnn_dim), act, Linear(gnn_dim, gnn_dim)))
        self.pool = global_mean_pool

        self.mlp = Sequential(
            Linear(gnn_dim + rdkit_dim, hidden_dim), act, Dropout(dropout_rate),
            Linear(hidden_dim, hidden_dim // 2), act, Dropout(dropout_rate),
            Linear(hidden_dim // 2, 1)
            )

    def forward(self, data):
        # encode atoms and bonds
        x = self.atom_encoder(data.x)
        edge_attr = self.bond_encoder(data.edge_attr)

        # GNN convolutions
        x = self.conv1(x, data.edge_index, edge_attr)
        x = self.conv2(x, data.edge_index, edge_attr)
        x = self.pool(x, data.batch)

        # handle RDKit features
        rdkit_feats = getattr(data, 'rdkit_feats', None)
        if rdkit_feats is not None:
            # Reshape the RDKit features tensor to be (batch_size, rdkit_dim)
            # The number of samples in the batch is given by x.shape[0] after pooling
            reshaped_rdkit_feats = rdkit_feats.view(x.shape[0], self.rdkit_dim)
            
            if x.shape[0] != reshaped_rdkit_feats.shape[0]:
                raise ValueError(f"Shape mismatch: GNN output ({x.shape[0]}) vs rdkit_feats ({reshaped_rdkit_feats.shape[0]})")
            
            x = torch.cat([x, reshaped_rdkit_feats], dim=1)
        else:
            raise ValueError("RDKit features not found in the data object")

        return self.mlp(x)

In [38]:
def train_hybrid_gnn(
    model: nn.Module,
    train_loader,
    val_loader,
    *,
    lr: float,
    optimizer: str = "Adam",
    weight_decay: float = 0.0,
    epochs: int = 100,
    patience: int = 10,
    save_dir: str = "saved_models/gnn",
    tag: str = "model",
    device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu"),
):
    os.makedirs(save_dir, exist_ok=True)
    model = model.to(device)
    if optimizer.lower() == "adamw":
        opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    else:
        opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    best, bad = float("inf"), 0
    best_path = os.path.join(save_dir, f"{tag}.pt")

    @torch.no_grad()
    def eval_once(loader):
        model.eval()
        preds, trues = [], []
        for b in loader:
            b = b.to(device)
            p = model(b)
            preds.append(p.cpu())
            trues.append(b.y.view(-1,1).cpu())
        preds = torch.cat(preds).numpy(); trues = torch.cat(trues).numpy()
        mse = np.mean((preds - trues)**2)
        return mse, preds, trues

    for ep in range(1, epochs+1):
        model.train()
        total, count = 0.0, 0
        for b in train_loader:
            b = b.to(device)
            pred = model(b)
            loss = F.mse_loss(pred, b.y.view(-1,1))
            opt.zero_grad(); loss.backward(); opt.step()
            total += loss.item() * b.num_graphs
            count += b.num_graphs
        tr_mse = total / max(1, count)
        va_mse, _, _ = eval_once(val_loader)
        print(f"Epoch {ep:02d} | Train MSE {tr_mse:.5f} | Val MSE {va_mse:.5f}")

        if va_mse < best - 1e-7:
            best, bad = va_mse, 0
            torch.save(model.state_dict(), best_path)
        else:
            bad += 1
            if bad >= patience:
                print("Early stopping.")
                break

    model.load_state_dict(torch.load(best_path, map_location=device))
    val_mse, val_pred, val_true = eval_once(val_loader)
    mae = np.mean(np.abs(val_pred - val_true))
    rmse = np.sqrt(val_mse)
    r2 = 1 - np.sum((val_pred - val_true)**2) / np.sum((val_true - val_true.mean())**2)
    print(f"[{tag}] Best Val — MAE {mae:.6f} | RMSE {rmse:.6f} | R2 {r2:.4f}")
    return model, best_path, {"MAE": mae, "RMSE": rmse, "R2": r2}


In [39]:
model_tg = HybridGNN(
    gnn_dim=128, rdkit_dim=X_tg_desc.shape[1], hidden_dim=256,
    dropout_rate=0.2, activation="ReLU"
)
model_tg, ckpt_tg, metrics_tg = train_hybrid_gnn(
    model_tg, train_loader_tg, val_loader_tg,
    lr=1e-3, optimizer="Adam", weight_decay=0.0,
    epochs=100, patience=10, save_dir="saved_models/gnn_tg", tag="hybridgnn_tg"
)

Epoch 01 | Train MSE 15444.07007 | Val MSE 11713.92578
Epoch 02 | Train MSE 11183.00550 | Val MSE 10380.86719
Epoch 03 | Train MSE 10230.25985 | Val MSE 8342.48535
Epoch 04 | Train MSE 9003.71527 | Val MSE 7855.45605
Epoch 05 | Train MSE 7574.64763 | Val MSE 5449.46729
Epoch 06 | Train MSE 6721.04160 | Val MSE 5008.34277
Epoch 07 | Train MSE 6470.01543 | Val MSE 5736.78174
Epoch 08 | Train MSE 6253.05764 | Val MSE 4692.91650
Epoch 09 | Train MSE 6326.17281 | Val MSE 4487.83496
Epoch 10 | Train MSE 6046.40858 | Val MSE 4503.41943
Epoch 11 | Train MSE 5916.84184 | Val MSE 4242.87891
Epoch 12 | Train MSE 6213.79196 | Val MSE 4538.29443
Epoch 13 | Train MSE 5540.59278 | Val MSE 4418.76318
Epoch 14 | Train MSE 5703.22842 | Val MSE 4029.75366
Epoch 15 | Train MSE 5410.52960 | Val MSE 4288.02002
Epoch 16 | Train MSE 5124.28050 | Val MSE 4578.30908
Epoch 17 | Train MSE 5743.87881 | Val MSE 4772.59766
Epoch 18 | Train MSE 5931.18839 | Val MSE 6021.19434
Epoch 19 | Train MSE 5849.53569 | Val MSE

C:\Users\mattg\AppData\Local\Temp\ipykernel_33644\3084240870.py:61: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_path, map_location=de

In [40]:
den_cfg = {'gnn_dim': 1024, 'hidden_dim': 384, 'dropout_rate': 0.3735260731607324,
           'lr': 5.956024201538505e-04, 'activation': 'Swish', 'optimizer': 'AdamW',
           'weight_decay': 8.619671341229739e-06}

model_den = HybridGNN(
    gnn_dim=den_cfg['gnn_dim'],
    rdkit_dim=X_density_desc.shape[1],
    hidden_dim=den_cfg['hidden_dim'],
    dropout_rate=den_cfg['dropout_rate'],
    activation=den_cfg['activation']
)
model_den, ckpt_den, metrics_den = train_hybrid_gnn(
    model_den, train_loader_den, val_loader_den,
    lr=den_cfg['lr'], optimizer=den_cfg['optimizer'],
    weight_decay=den_cfg['weight_decay'],
    epochs=120, patience=15,  
    save_dir="saved_models/gnn_density", tag="hybridgnn_density"
)


Epoch 01 | Train MSE 2.17636 | Val MSE 0.06613
Epoch 02 | Train MSE 0.38810 | Val MSE 0.03597
Epoch 03 | Train MSE 0.12518 | Val MSE 0.02702
Epoch 04 | Train MSE 0.07666 | Val MSE 0.02018
Epoch 05 | Train MSE 0.06099 | Val MSE 0.03023
Epoch 06 | Train MSE 0.05001 | Val MSE 0.01728
Epoch 07 | Train MSE 0.04629 | Val MSE 0.02006
Epoch 08 | Train MSE 0.03975 | Val MSE 0.01823
Epoch 09 | Train MSE 0.03617 | Val MSE 0.01693
Epoch 10 | Train MSE 0.03190 | Val MSE 0.01054
Epoch 11 | Train MSE 0.03053 | Val MSE 0.01393
Epoch 12 | Train MSE 0.03381 | Val MSE 0.01481
Epoch 13 | Train MSE 0.02940 | Val MSE 0.01107
Epoch 14 | Train MSE 0.02420 | Val MSE 0.01387
Epoch 15 | Train MSE 0.03742 | Val MSE 0.01703
Epoch 16 | Train MSE 0.02867 | Val MSE 0.01163
Epoch 17 | Train MSE 0.02569 | Val MSE 0.01301
Epoch 18 | Train MSE 0.02469 | Val MSE 0.00796
Epoch 19 | Train MSE 0.02491 | Val MSE 0.00744
Epoch 20 | Train MSE 0.02079 | Val MSE 0.00660
Epoch 21 | Train MSE 0.02265 | Val MSE 0.01368
Epoch 22 | Tr

C:\Users\mattg\AppData\Local\Temp\ipykernel_33644\3084240870.py:61: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_path, map_location=de

In [ ]:
# ===== Submission: predict each target on test set and combine =====
import os, numpy as np, pandas as pd, joblib, torch
from tqdm.auto import tqdm
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors, Descriptors, DataStructs
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as GeoDataLoader

label_cols = ['Tg','FFV','Tc','Density','Rg']

# ---- 0) Load test ids & smiles (and sample for dtype of id) ----
sample = pd.read_csv(os.path.join(DATA_ROOT, 'sample_submission.csv'))
test_df = pd.read_csv(os.path.join(DATA_ROOT, 'test.csv'))
test_ids = test_df['id'].astype(sample['id'].dtype).values
test_smiles = test_df['SMILES'].astype(str).tolist()

# ---- 1) Helpers for features ----
def canonicalize(s):
    try:
        # you already have canonicalize_polymer_smiles; use that if imported:
        return canonicalize_polymer_smiles(s)
    except NameError:
        # fallback: plain RDKit canonicalization
        m = Chem.MolFromSmiles(s)
        return Chem.MolToSmiles(m) if m is not None else ""

def morgan_bits_from_smiles(smiles_list, n_bits=1024, radius=3):
    """Return (N, n_bits) uint8 array of Morgan bit vectors; no dropping."""
    fps = np.zeros((len(smiles_list), n_bits), dtype=np.uint8)
    for i, smi in enumerate(smiles_list):
        bv = np.zeros((n_bits,), dtype=np.uint8)
        try:
            cs = canonicalize(smi)
            m = Chem.MolFromSmiles(cs)
            if m is not None:
                fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(m, radius=radius, nBits=n_bits)
                DataStructs.ConvertToNumpyArray(fp, bv)
        except Exception:
            pass  # leave zeros if it fails
        fps[i] = bv
    return fps

def compute_rdkit_descriptors(m):
    s = canonicalize_polymer_smiles(m)
    m = Chem.MolFromSmiles(s)
    heavy = Descriptors.HeavyAtomCount(m)
    arom_atoms = sum(a.GetIsAromatic() for a in m.GetAtoms())
    hetero = sum(a.GetAtomicNum() not in (1,6) for a in m.GetAtoms())
    aromatic_prop = (arom_atoms / max(1, heavy))
    hetero_frac   = (hetero / max(1, heavy))

    return [
        Descriptors.MolMR(m),                    # 1
        Descriptors.MolLogP(m),                  # 2 (SlogP)
        rdmd.CalcFractionCSP3(m),                # 3
        rdmd.CalcNumAromaticRings(m),            # 4
        aromatic_prop,                           # 5
        Descriptors.TPSA(m),                     # 6
        Descriptors.NumRotatableBonds(m),        # 7
        hetero_frac,                             # 8
        rdmd.CalcLabuteASA(m),                   # 9
        # rdmd.CalcBalabanJ(m),                    # 10
        rdmd.CalcKappa1(m),                      # 11
        rdmd.CalcKappa2(m),                      # 12
    ]


def build_desc_matrix(smiles_list, fill_values=None):
    X = np.array([compute_rdkit_descriptors(s) for s in smiles_list], dtype=float)
    mask = ~np.isfinite(X)  # NaN/Inf
    if fill_values is None:
        # fallback: column-wise median
        col_med = np.nanmedian(np.where(mask, np.nan, X), axis=0)
        fill_values = np.where(np.isfinite(col_med), col_med, 0.0)
    X[mask] = np.take(fill_values, np.where(mask)[1])
    return X, fill_values

def smiles2graph_safe(smi):
    g = smiles2graph(smi)  # your utility
    return Data(
        x=torch.tensor(g['node_feat'], dtype=torch.long),
        edge_index=torch.tensor(g['edge_index'], dtype=torch.long),
        edge_attr=torch.tensor(g['edge_feat'], dtype=torch.long),
    )

def make_pyg_dataset_for_infer(smiles_list, X_desc):
    data_list = []
    for i, smi in enumerate(smiles_list):
        try:
            d = smiles2graph_safe(smi)
            d.rdkit_feats = torch.tensor(X_desc[i], dtype=torch.float32)
            # dummy y to satisfy code paths; not used at inference
            d.y = torch.tensor([0.0], dtype=torch.float32)
            data_list.append(d)
        except Exception:
            # on failure, create a 1-node empty-ish graph as fallback
            x = torch.zeros((1,1), dtype=torch.long)
            edge_index = torch.zeros((2,0), dtype=torch.long)
            edge_attr = torch.zeros((0,1), dtype=torch.long)
            d = Data(x=x, edge_index=edge_index, edge_attr=edge_attr,
                     rdkit_feats=torch.tensor(X_desc[i], dtype=torch.float32),
                     y=torch.tensor([0.0], dtype=torch.float32))
            data_list.append(d)
    return data_list

def predict_gnn(model, loader, device):
    model.eval()
    outs = []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            pred = model(batch)  # [B,1]
            outs.append(pred.squeeze(1).detach().cpu().numpy())
    return np.concatenate(outs, axis=0)

# ---- 2) RF predictions (FFV, Tc, Rg) ----
# Use the same FP config you trained with
FP_BITS, FP_RADIUS = 1024, 3
X_test_fp = morgan_bits_from_smiles(test_smiles, n_bits=FP_BITS, radius=FP_RADIUS)

rf_ffv_bundle = joblib.load(p_ffv); rf_ffv = rf_ffv_bundle['model']
rf_tc_bundle  = joblib.load(p_tc);  rf_tc  = rf_tc_bundle['model']
rf_rg_bundle  = joblib.load(p_rg);  rf_rg  = rf_rg_bundle['model']

pred_ffv = rf_ffv.predict(X_test_fp).astype(float)
pred_tc  = rf_tc.predict(X_test_fp).astype(float)
pred_rg  = rf_rg.predict(X_test_fp).astype(float)

# ---- 3) GNN predictions (Tg, Density) ----
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 3a) Tg descriptors for test (fill with TRAIN stats)
# Assumes X_tg_desc exists from training; else set to None to use medians
tg_fill = np.nanmean(X_tg_desc, axis=0) if 'X_tg_desc' in globals() else None
X_tg_desc_test, _ = build_desc_matrix(test_smiles, fill_values=tg_fill)

data_tg = make_pyg_dataset_for_infer(test_smiles, X_tg_desc_test)
test_loader_tg = GeoDataLoader(data_tg, batch_size=256, shuffle=False, num_workers=0, pin_memory=True)

model_tg_infer = HybridGNN(
    gnn_dim=128,
    rdkit_dim=X_tg_desc_test.shape[1],
    hidden_dim=256,
    dropout_rate=0.2,
    activation="ReLU",
).to(device)
model_tg_infer.load_state_dict(torch.load(ckpt_tg, map_location=device))
pred_tg = predict_gnn(model_tg_infer, test_loader_tg, device)

# 3b) Density descriptors for test (fill with TRAIN stats)
den_fill = np.nanmean(X_density_desc, axis=0) if 'X_density_desc' in globals() else None
X_den_desc_test, _ = build_desc_matrix(test_smiles, fill_values=den_fill)

data_den = make_pyg_dataset_for_infer(test_smiles, X_den_desc_test)
test_loader_den = GeoDataLoader(data_den, batch_size=256, shuffle=False, num_workers=0, pin_memory=True)

# your tuned density config
den_cfg = {'gnn_dim': 1024, 'hidden_dim': 384, 'dropout_rate': 0.3735260731607324,
           'lr': 5.956024201538505e-04, 'activation': 'Swish', 'optimizer': 'AdamW',
           'weight_decay': 8.619671341229739e-06}

model_den_infer = HybridGNN(
    gnn_dim=den_cfg['gnn_dim'],
    rdkit_dim=X_den_desc_test.shape[1],
    hidden_dim=den_cfg['hidden_dim'],
    dropout_rate=den_cfg['dropout_rate'],
    activation=den_cfg['activation'],
).to(device)
model_den_infer.load_state_dict(torch.load(ckpt_den, map_location=device))
pred_density = predict_gnn(model_den_infer, test_loader_den, device)

# Safety: replace any NaNs/Infs with finite numbers
pred_tg       = np.nan_to_num(pred_tg, nan=0.0, posinf=0.0, neginf=0.0)
pred_density  = np.nan_to_num(pred_density, nan=0.0, posinf=0.0, neginf=0.0)
pred_ffv      = np.nan_to_num(pred_ffv, nan=0.0, posinf=0.0, neginf=0.0)
pred_tc       = np.nan_to_num(pred_tc, nan=0.0, posinf=0.0, neginf=0.0)
pred_rg       = np.nan_to_num(pred_rg, nan=0.0, posinf=0.0, neginf=0.0)

# ---- 4) Assemble submission in required order ----
sub = pd.DataFrame({
    'id': test_ids,
    'Tg': pred_tg,
    'FFV': pred_ffv,
    'Tc': pred_tc,
    'Density': pred_density,
    'Rg': pred_rg,
})

# Optional: clip to train ranges to avoid wild outliers
if 'train_df' in globals():
    for k in label_cols:
        lo = np.nanmin(train_df[k].values)
        hi = np.nanmax(train_df[k].values)
        sub[k] = np.clip(sub[k].values, lo, hi)

# Save
sub.to_csv('submission.csv', index=False)
print('Submission file created')
sub.head()

Submission file created


C:\Users\mattg\AppData\Local\Temp\ipykernel_33644\2426260556.py:137: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_tg_infer.load_state_dict(torch.load(ckpt_tg, map_loc

,id,Tg,FFV,Tc,Density,Rg
0,1109053969,130.776535,0.387567,0.208534,1.150190,18.997804
1,1422188626,174.627182,0.362225,0.228302,1.069831,19.638581
2,2032016830,59.418404,0.367133,0.202228,1.072706,18.213128


# Conclusions

## Model Performance Summary

All baseline models were initially trained and evaluated on a 5,000 molecule subset of the full dataset. Below is a comparison of results across different featurization strategies and model types:

### 2D Baseline Models

| Model Type    | Featurization      | MAE   | RMSE  | R²    | Notes                                 |
| ------------- | ------------------ | ----- | ----- | ----- | ------------------------------------- |
| MLP (Tuned)   | RDKit Fingerprints | 0.426 | 0.574 | 0.798 | Strong performance across all metrics |
| KRR (Tuned)   | RDKit Fingerprints | 0.454 | 0.593 | 0.784 | Good overall, slightly lower R²       |
| RF (Tuned)    | RDKit Fingerprints | 0.423 | 0.583 | 0.791 | Best MAE, very competitive overall    |
| MLP (Tuned)   | Coulomb Matrix     | 0.636 | 0.819 | 0.588 | Significantly weaker performance      |
| MLP (Untuned) | RDKit Fingerprints | 0.467 | 0.609 | 0.772 | Solid untuned baseline                |
| KRR (Untuned) | RDKit Fingerprints | 0.519 | 0.668 | 0.726 | Notable drop from tuned version       |
| RF (Untuned)  | RDKit Fingerprints | 0.426 | 0.587 | 0.788 | Surprisingly close to tuned RF        |
| MLP (Untuned) | Coulomb Matrix     | 0.663 | 0.847 | 0.559 | Consistently underperforms            |

### Graph Neural Network Models (ChemML)

| Model Type    | Featurization               | MAE   | RMSE  | R²    | Notes                                |
| ------------- | --------------------------- | ----- | ----- | ----- | ------------------------------------ |
| GNN (Tuned)   | `tensorise_molecules` Graph | 0.302 | 0.411 | 0.900 | Best results from ChemML experiments |
| GNN (Untuned) | `tensorise_molecules` Graph | 0.400 | 0.519 | 0.841 | Strong but less optimized            |

### Final Hybrid GNN Model Trained on Full Dataset (OGB-Compatible)

| Model Type           | Featurization                          | MAE   | RMSE  | R²    | Notes                              |
| -------------------- | -------------------------------------- | ----- | ----- | ----- | ---------------------------------- |
| Hybrid GNN (Tuned)   | OGB `smiles2graph` + RDKit descriptors | 0.159 | 0.234 | 0.965 | State-of-the-art level performance |
| Hybrid GNN (Untuned) | OGB `smiles2graph` + RDKit descriptors | 0.223 | 0.308 | 0.939 | Still very strong pre-tuning       |

---

## Model Error Analysis

I performed qualitative evaluation by comparing predicted vs. true HOMO–LUMO gaps for both randomly selected and poorly predicted molecules. The worst performing molecules often showed rare or complex structures likely underrepresented in the training set. This highlights the importance of structural diversity and potentially more expressive 3D information to improve generalization.

## Next Steps: Integrating 3D Molecular Information

To push performance even further and overcome limitations of 2D graphs and hand crafted descriptors, my next step will involve:

* Using **3D molecular geometries** 
* Incorporating **interatomic distances**, angles, and **spatial encoding** (SchNet, DimeNet, or SE(3)-equivariant models)
* Comparing results against the current best MAE (\~0.159)

This direction aligns with trends in molecular property prediction where 3D aware models often outperform purely 2D approaches, especially for quantum properties like HOMO–LUMO gaps.
